In [45]:
# Add project root to Python path
import os
import sys

# Get the absolute path to the project root directory
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the project root to Python path if it's not already there
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Verify the path is added correctly
print(f"Added project root to Python path: {project_root}")

Added project root to Python path: /Users/brylie/code/remote-sensing-analysis


In [46]:
import geemap
import ee
from config.areas import get_area
from src.extractors.sentinel import get_sentinel_data
from src.metrics.vegetation import calculate_evi, calculate_lai
from src.metrics.moisture import calculate_msi
from src.processors.preprocessing import add_date

Original source: https://github.com/krishnakafle/Blog_kaflekrishna/blob/main/GEE/LAI/LAI_calculations.ipynb

In [47]:
ee.Initialize()

In [48]:
# Using get_area function from config.areas module to get Finland geometry
area_of_interest = get_area("finland")

In [49]:
START_DATE = ee.Date("2024-10-01")
END_DATE = ee.Date("2024-11-01")

# Using get_sentinel_data function from src.extractors.sentinel module
sentinel_data = get_sentinel_data(START_DATE, END_DATE, area_of_interest)

# Apply EVI calculation to the fetched data
sentinel_data_with_evi = sentinel_data.map(calculate_evi)

In [50]:
# Apply LAI calculation and add MSI calculation
processed_data = (
    sentinel_data_with_evi
    .map(calculate_lai)
    .map(calculate_msi)
    .map(add_date)
)

# Create composite images for visualization
composite_image = (
    processed_data
    .median()
    .clip(area_of_interest)
)

## Visualizing Vegetation and Moisture Metrics

Below we'll visualize two key metrics:
- Leaf Area Index (LAI): Higher values indicate more leaf material and denser vegetation
- Moisture Stress Index (MSI): Higher values indicate greater vegetation water stress

Both metrics provide complementary information about vegetation health and status.

In [51]:
# Set up color palettes for visualization
veg_palette = [
    "FFFFFF", "CE7E45", "DF923D", "F1B555", "FCD163", "99B718",
    "74A901", "66A000", "529400", "3E8601", "207401", "056201",
    "004C00", "023B01", "012E01", "011D01", "011301"
]

# LAI visualization parameters (higher = greener)
lai_params = {"min": 0.5, "max": 3.5, "palette": veg_palette}

# MSI visualization parameters (higher = more moisture stress)
msi_params = {"min": 0.4, "max": 1.4, "palette": veg_palette[::-1]}

# Create map
map = geemap.Map()
map.centerObject(area_of_interest, 8)

# Add LAI layer
map.addLayer(composite_image.select("LAI"), lai_params, "Leaf Area Index (LAI)")

# Add MSI layer
map.addLayer(composite_image.select("MSI"), msi_params, "Moisture Stress Index (MSI)")

# Add layer controls and display the map
map.addLayerControl()
map

Map(center=[64.23487926926077, 26.152236411247713], controls=(WidgetControl(options=['position', 'transparent_…

In [52]:
# Export a side-by-side comparison of LAI and MSI
import matplotlib.pyplot as plt
from IPython.display import display

# Get images as numpy arrays for plotting
region = area_of_interest.bounds().getInfo()['coordinates']
scale = 5000  # Use a coarse scale for quick visualization

# Function to fetch a band as an array
def get_band_array(image: ee.Image, band_name: str) -> dict:
    return image.select(band_name).getThumbURL({
        'min': lai_params['min'] if band_name == 'LAI' else msi_params['min'],
        'max': lai_params['max'] if band_name == 'LAI' else msi_params['max'],
        'palette': lai_params['palette'] if band_name == 'LAI' else msi_params['palette'],
        'dimensions': 500,
        'region': region
    })

# Create URLs for both metrics
lai_url = get_band_array(composite_image, 'LAI')
msi_url = get_band_array(composite_image, 'MSI')

print(f"LAI visualization URL: {lai_url}")
print(f"MSI visualization URL: {msi_url}")

LAI visualization URL: https://earthengine.googleapis.com/v1/projects/400372326428/thumbnails/ae5f80a65af33011c20e9dcc7c99831d-904d7236e53e6ae398bb91f884557080:getPixels
MSI visualization URL: https://earthengine.googleapis.com/v1/projects/400372326428/thumbnails/b118b42a532e23aedcbf138cb603223d-a1b8433c77071fabd4440659572ad48a:getPixels
